In [1]:
import pandas as pd
import numpy as np

In [2]:
def categorize_dti_cat(row):
    dti = row['debt_to_income_ratio']
    healthy = ['<20%', '20%-<30%', '30%-<36%' ]
    manageable = ['36', '37', '38', '39', '40', '41', '42']
    unmanageable = ['43', '44', '45', '46', '47', '48', '49']
    struggling = ['50%-60%', '>60%']
    if dti in healthy:
        return '1'
    elif dti in manageable:
        return '2'
    elif dti in unmanageable:
        return '3'
    elif dti in struggling:
        return '4'     
    elif dti == 'Exempt':
        return '5'
    elif dti == 'null':
        return '6'    
        
def categorize_cltv(row):
    cltv = row['cltv_ratio']
    if cltv <= 80:
        return '1'
    elif cltv > 80:
        return '2'       
    else:
        return '3'

def categorize_age(row):
    age = row['applicant_age']
    if age == '<25':
        return '1'
    elif age == '25-34':
        return '2'
    elif age == '35-44':
        return '3'
    elif age == '45-54':
        return '4'
    elif age == '55-64':
        return '5'
    elif age == '65-74':
        return '6'
    elif age == '>74':
        return '7'
    elif age == '8888' or age == '9999':
        return '8'
    
def categorize_sex(row):
    sex = row['applicant_sex']
    if sex == '1':
        return '1'
    elif sex == '2':
        return '2'
    elif sex == '3' or sex == '4':
        return '3'
    elif sex == '6':
        return '4'
        
        
def categorize_aus(row):
    aus_cat = row['aus_1']
    if aus_cat == '1':
        return '1'
    elif aus_cat  == '2':
        return '2'
    elif aus_cat == '3':
        return '3'
    elif aus_cat  == '4':
        return '4'
    elif aus_cat  == '5':
        return '5'
    elif aus_cat == '6' or '1111':
        return '6'
    
def categorize_loan_term(row):
    loan_term = row['em_loan_term']
    if loan_term == 360:
        return '1'
    elif loan_term < 360:
        return '2'
    elif loan_term > 360:
        return '3'
    else:
        return '4'
        
        
def categorize_lmi(row):
    tract_msa_ratio = row['tract_msa_ratio']
    if tract_msa_ratio > 0 and tract_msa_ratio < 50:
        return '1'
    elif tract_msa_ratio >= 50 and tract_msa_ratio < 80:
        return '2'
    elif tract_msa_ratio >= 80 and tract_msa_ratio < 120:
        return '3'
    elif tract_msa_ratio >= 120:
        return '4'
    elif tract_msa_ratio == 0:
        return '5'

def categorize_race(row):
    race = row['applicant_race_1']
    asian = ['2.0', '21.0' , '22.0', '23.0' , '24.0' , '25.0' , '26.0' , '27.0']
    pac = ['4.0' , '41.0' , '42.0' , '43.0' , '44.0']
    na = ['6.0' , '7.0']
    if race == '1.0':
        return '1'
    elif race in asian:
        return '2'
    elif race == '3.0':
        return '3'
    elif race in pac:
        return '4'
    elif race == '5.0':
        return '5'
    elif race in na:
        return '6'

def categorize_credit(row):
    equifax = ['1']
    experian = ['2']
    transunion = ['3', '4']
    vantage = ['5', '6']
    more_than_one = ['7']
    other_model = ['8']
    credit_na = ['9', '1111']
    credit_model = row['applicant_credit_score_type']
    if credit_model in equifax:
        return '1'
    elif credit_model in experian:
        return '2'
    elif credit_model in transunion:
        return '3'
    elif credit_model in vantage:
        return '4'
    elif credit_model in more_than_one:
        return '5'
    elif credit_model in other_model:
        return '6'
    elif credit_model in credit_na:
        return '7'

# Import Data

In [3]:
clean_df = pd.read_csv("cleaned.csv", dtype = str)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 71 columns):
 #   Column                                    Dtype 
---  ------                                    ----- 
 0   activity_year                             object
 1   lei                                       object
 2   derived_msa_md                            object
 3   state_code                                object
 4   county_code                               object
 5   census_tract                              object
 6   conforming_loan_limit                     object
 7   derived_loan_product_type                 object
 8   derived_dwelling_category                 object
 9   derived_ethnicity                         object
 10  derived_race                              object
 11  derived_sex                               object
 12  action_taken                              object
 13  purchaser_type                            object
 14  preapproval       

# Categorize Debt-to-Income Ratio

In [4]:
dti_df = pd.DataFrame(clean_df['debt_to_income_ratio'].value_counts(dropna = False))
dti_df = dti_df.reset_index().rename(columns = {'index': 'debt_to_income_ratio', 'debt_to_income_ratio': 'count'})
dti_df = dti_df.fillna('null')
dti_df['dti_cat'] = dti_df.apply(categorize_dti_cat, axis = 1)
dti_df = dti_df.drop(columns = ['count'], axis = 1)
dti_df = dti_df.replace('null', np.nan)
clean_df = pd.merge(clean_df, dti_df, how = 'left', on = ['debt_to_income_ratio'])

Categorize Loan-to-Value Ratio

In [5]:
cltv_df = pd.DataFrame(clean_df['combined_loan_to_value_ratio'].value_counts(dropna = False))
cltv_df = cltv_df.reset_index().rename(columns = {'index': 'combined_loan_to_value_ratio', 'combined_loan_to_value_ratio': 'count'})
cltv_df.loc[(cltv_df['combined_loan_to_value_ratio'] != 'Exempt'), 'cltv_ratio'] = cltv_df['combined_loan_to_value_ratio']
cltv_df['cltv_ratio'] = pd.to_numeric(cltv_df['cltv_ratio'])
cltv_df['downpayment_flag'] = cltv_df.apply(categorize_cltv, axis = 1)
cltv_df = cltv_df.drop(columns = ['count', 'cltv_ratio'], axis = 1)
clean_df = pd.merge(clean_df, cltv_df, how = 'left', on = ['combined_loan_to_value_ratio'])

# Categorize Applicant Age

In [6]:
age_df = pd.DataFrame(clean_df['applicant_age'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'applicant_age', 'applicant_age': 'count'})
age_df['applicant_age_cat'] = age_df.apply(categorize_age, axis = 1)
age_df = age_df.drop(columns = ['count'], axis = 1)
clean_df = pd.merge(clean_df, age_df, how = 'left', on = ['applicant_age'])

# Categorize Applicant Sex

In [7]:
sex_df = pd.DataFrame(clean_df['applicant_sex'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'applicant_sex', 'applicant_sex': 'count'})
sex_df = sex_df.drop(columns = ['count'], axis = 1)
sex_df['applicant_sex_cat'] = sex_df.apply(categorize_sex, axis = 1)
clean_df = pd.merge(clean_df, sex_df, how = 'left', on = ['applicant_sex'])

# Categorize Automated Underwriting Systems

In [8]:
aus_df = pd.DataFrame(clean_df['aus_1'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'aus_1', 'aus_1': 'count'})
aus_df = aus_df.drop(columns = ['count'], axis = 1)
aus_df['aus_cat'] = aus_df.apply(categorize_aus, axis = 1)
clean_df = pd.merge(clean_df, aus_df, how = 'left', on = ['aus_1'])

# Categorize Loan Term

In [9]:
loanterm_df = pd.DataFrame(clean_df['loan_term'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'loan_term', 'loan_term': 'count'})
loanterm_df.loc[(loanterm_df['loan_term'] != 'Exempt'), 'em_loan_term'] = loanterm_df['loan_term']
loanterm_df['em_loan_term'] = pd.to_numeric(loanterm_df['em_loan_term'])
loanterm_df['mortgage_term'] = loanterm_df.apply(categorize_loan_term, axis = 1)
loanterm_df = loanterm_df.drop(columns = ['count', 'em_loan_term'])
clean_df = pd.merge(clean_df, loanterm_df, how = 'left', on = ['loan_term'])

# Categorize MSA Income Percentage

In [10]:
tractmsa_income_df = pd.DataFrame(clean_df['tract_to_msa_income_percentage'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'tract_to_msa_income_percentage', 'tract_to_msa_income_percentage': 'count'})
tractmsa_income_df['tract_msa_ratio'] = pd.to_numeric(tractmsa_income_df['tract_to_msa_income_percentage'])
tractmsa_income_df['lmi_def'] = tractmsa_income_df.apply(categorize_lmi, axis = 1)
tractmsa_income_df = tractmsa_income_df.drop(columns = ['count', 'tract_msa_ratio'], axis = 1)
clean_df = pd.merge(clean_df, tractmsa_income_df, how = 'left', on = ['tract_to_msa_income_percentage'])

# Categorize Applicant Race

In [11]:
race_df = pd.DataFrame(clean_df['applicant_race_1'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'applicant_race_1', 'applicant_race_1': 'count'})
race_df['applicant_race_cat'] = race_df.apply(categorize_race, axis = 1)
race_df = race_df.drop(columns = ['count'], axis = 1)
clean_df = pd.merge(clean_df, race_df, how = 'left', on = ['applicant_race_1'])

# Categorize Applicant Credit

In [12]:
credit_df = pd.DataFrame(clean_df['applicant_credit_score_type'].value_counts(dropna = False)).reset_index().rename(columns = {'index': 'applicant_credit_score_type', 'applicant_credit_score_type': 'count'})
credit_df['applicant_credit_cat'] = credit_df.apply(categorize_credit, axis = 1)
credit_df = credit_df.drop(columns = ['count'], axis = 1)
clean_df = pd.merge(clean_df, credit_df, how = 'left', on = ['applicant_credit_score_type'])

# Log Transformation of Income and Loan Amount

In [13]:
clean_df['income'] = pd.to_numeric(clean_df['income'])
clean_df['loan_amount'] = pd.to_numeric(clean_df['loan_amount'])
clean_df['income_log'] = np.log(clean_df['income'])
clean_df['loan_log'] = np.log(clean_df['loan_amount'])

/Users/rutuja/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/rutuja/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Filter

In [14]:
l = ['1', '2', '3', '4']
clean_df = clean_df[((clean_df['loan_type'] == '1')) &\
                        (clean_df['income'] > 0) &\
                        (clean_df['occupancy_type'] == '1') &\
                        (clean_df['total_units'].isin(l)) &\
                        (clean_df['loan_purpose'] == '1') &\
                        ((clean_df['action_taken'] == '1')|(clean_df['action_taken'] == '3')) &\
                        (clean_df['construction_method'] == '1') &\
                        (clean_df['lien_status'] == '1') &\
                        (clean_df['business_or_commercial_purpose'] != '1') &\
                        ((clean_df['dti_cat'] == '1')|(clean_df['dti_cat'] == '2')|(clean_df['dti_cat'] == '3')|(clean_df['dti_cat'] == '4')) &\
                        (clean_df['downpayment_flag'] != '3') &\
                        (clean_df['applicant_age_cat'] != '8') &\
                        ((clean_df['applicant_sex_cat'] == '1')|(clean_df['applicant_sex_cat'] == '2')) &\
                        ((clean_df['aus_cat'] != '5')|(clean_df['aus_cat'] != '6')) &\
                        (clean_df['mortgage_term'] != '4') &\
                        (clean_df['applicant_age_cat'] != '8') &\
                        ((clean_df['applicant_race_cat'] == '1')|(clean_df['applicant_race_cat'] == '2')|(clean_df['applicant_race_cat'] == '3')|(clean_df['applicant_race_cat'] == '4')|(clean_df['applicant_race_cat'] == '5'))].copy()

# Write to CSV

In [15]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659889 entries, 24 to 5999974
Data columns (total 82 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   activity_year                             659889 non-null  object 
 1   lei                                       659889 non-null  object 
 2   derived_msa_md                            659889 non-null  object 
 3   state_code                                656025 non-null  object 
 4   county_code                               655570 non-null  object 
 5   census_tract                              655288 non-null  object 
 6   conforming_loan_limit                     659889 non-null  object 
 7   derived_loan_product_type                 659889 non-null  object 
 8   derived_dwelling_category                 659889 non-null  object 
 9   derived_ethnicity                         659889 non-null  object 
 10  derived_race      

In [16]:
clean_df.to_csv('categorized_data.csv', index = False)